# 6.2. Simulación para medición de riesgos (VaR).

 - Value at Risk (VaR): es una estadística que cuantifica el límite de las posibles pérdidas dentro de una cartera, o activo, durante un período de tiempo específico y a un nivel de probabilidad dado.  
 - Expected Shortfall: Media de los retornos que están por debajo del VaR.

<center>
<img src="imgs/var_es.png"  alt="drawing" width="500"/>
</center>

- Objetivos: 
    - Utilizando la serie de precios del SP500 de los últimos 5 años (ticker SPY en IEX), calcula:
    - Un gráfico con el drawdown.
    - Pinta la distibución de los retornos usando la función distplot, de seaborn.
    - Value at Risk (VaR) historico con un nivel de cofianza del 0.05 (usa la función quantile sobre los retornos).
    - Expected Shortfall (ES): media de los retornos que están por debajo del VaR.

### Obtenemos los datos de SPY de IEX

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [ ]:
TOKEN = 'TU_TOKEN'
base_url = 'https://cloud.iexapis.com/stable' 

In [ ]:
def get_data_iex(symbol, st_range='1y'):
    
    url = f"{base_url}/stock/{symbol}/chart/{st_range}"
    r = requests.get(url, params={'token': TOKEN})
    data = pd.DataFrame(r.json())
    data = data.set_index('date')
    data.name = symbol
    return data

In [ ]:
data_day = get_data_iex('SPY', st_range='5y')

In [ ]:
close_spy = data_day.close
close_spy.head()

In [ ]:
close_spy.plot()

### Calculamos y graficamos el Drawdown

In [ ]:
close_spy.plot()
close_spy.expanding().max().plot()

In [ ]:
draw_down = (close_spy/close_spy.expanding().max()) - 1

In [ ]:
draw_down.plot()

### Calculamos y pintamos la distribución de los retornos

In [ ]:
import seaborn as sns

In [ ]:
retornos = np.log(close_spy).diff().dropna()

In [ ]:
sns.displot(retornos, rug=True)

### Value at Risk (VaR, método histórico) and Expected Shortfall (ES)

El VaR es el valor límite de pérdida, sobre una probabilidad y un horizonte.

Por ejemplo, un VaR al 95% significa que un 5% de los retornos están por debajo de ese valor límite (del VaR).

El VaR NO es una pérdida máxima. Es un umbral, en función de una probabilidad y un horizonte temporal.

Expected Shortfall (ES): media de los retornos que están por debajo del VaR.

In [ ]:
var = retornos.quantile(q=0.05)
shortfall = retornos[retornos < var].mean()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
sns.histplot(retornos, ax=ax)
sns.rugplot(retornos, ax=ax)
ax.axvline(var, color='r')
ax.axvline(shortfall, color='k')